In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread
import os
import tensorflow_hub as hub
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Loading the model and EDA

In [11]:
train_path = '../input/car-damage-detection/data1a/training/'
test_path = '../input/car-damage-detection/data1a/validation/'

dim1 = []
dim2 = []
for x in os.listdir(test_path+'01-whole'):
    d1,d2,colors = imread(test_path+'01-whole/'+x).shape
    dim1.append(d1)
    dim2.append(d2)

sns.jointplot(x=dim1,y=dim2);

# Data preperation

For augmentation of data imagedatagenerator was used, gave parameters such as rotation range, width shift range, horizontal flip and some other values to generate more images the model could train on. 

In [12]:
input_shape = (224,224,3)    # this is the input shape specifications of the mobilenet model.

image_gen = ImageDataGenerator(rotation_range=20,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              shear_range=0.1,
                              zoom_range=0.1,
                              horizontal_flip=True,
                              fill_mode='nearest',
                              rescale=1/255)

image_gen.flow_from_directory(train_path)

In [13]:
image_gen.flow_from_directory(test_path)

In [14]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss',patience=2)

batch_size=16

train_image_gen = image_gen.flow_from_directory(train_path,
                                               target_size=input_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='sparse',
                                               )

In [15]:
test_image_gen = image_gen.flow_from_directory(test_path,
                                               target_size=input_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='sparse',
                                              shuffle=False)

In [29]:
train_image_gen.class_indices

# Model creation

Created model using transfer learning, the pretrained model used here is mobilenet version 3.

Added on output layer on top of the pretrained layer to create the model

In [16]:
feature_extractor_model = "https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5"

model_without_top_layer = hub.KerasLayer(
feature_extractor_model,input_shape=(224,224,3),trainable=False);

model = tensorflow.keras.Sequential([
    model_without_top_layer,
    tensorflow.keras.layers.Dense(1,activation='sigmoid')
]);

model.summary()

In [17]:
model.compile(
    loss = tensorflow.keras.losses.BinaryCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy'])

In [18]:
model.fit(train_image_gen,epochs=20,batch_size=16,
          validation_data=test_image_gen,
          callbacks=[early_stop]);

# Saving the weights

In [19]:
from tensorflow.keras.models import save_model

In [20]:
model.save('carscan.h5')

# Evaluating the model

Since this being a calssification problem the evaluation method used were confusion matrix, classification matrix and accuracy score.

In [21]:
model.evaluate(test_image_gen)

In [22]:
model.metrics_names

In [23]:
pred = model.predict(test_image_gen)

In [24]:
predictions = pred>0.5

In [25]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [26]:
print(confusion_matrix(test_image_gen.classes,predictions))

In [27]:
print(classification_report(test_image_gen.classes,predictions))

In [28]:
print(accuracy_score(test_image_gen.classes,predictions))